In [76]:
import requests
import json
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import sys, getopt, pprint
from pymongo import MongoClient

In [77]:
tests_day_wise = pd.read_csv('https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv')


tests_day_wise.drop(columns = tests_day_wise.columns[3:],axis=1,inplace=True)
tests_day_wise.drop(columns = tests_day_wise.columns[0],axis=1,inplace=True)

tests_day_wise = tests_day_wise.fillna(0)

tests_day_wise["Total Samples Tested"] = tests_day_wise["Total Samples Tested"].astype(int)

tests_day_wise.columns = ["Date","Total Tested"]


tests_day_wise.to_csv('./test_data/test_data/tests_day_wise.csv', index=False)

# tests_day_wise.tail(10)

In [79]:
csvfile = open('./test_data/test_data/tests_day_wise.csv', 'r')
reader = csv.DictReader( csvfile )
mongo_client=MongoClient() 
db=mongo_client.tests_day_wise
db.segment.drop()
header = tests_day_wise.columns 

for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    db.segment.insert(row)

In [80]:
tests_state_wise = pd.read_csv('https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv')

tests_state_wise.to_csv('./data/tests_state_wise.csv', index=False)

tests_state_wise.drop(columns=tests_state_wise.columns[5:],axis=1,inplace=True)
tests_state_wise.drop(columns="Tag (Total Tested)",axis=1,inplace=True)

tests_state_wise = tests_state_wise.fillna(0)
tests_state_wise = tests_state_wise[tests_state_wise["Total Tested"]!="Not Available"]
tests_state_wise = tests_state_wise.reset_index(drop=True)


tests_state_wise

,Updated On,State,Total Tested,Positive
0,17/04/2020,Andaman and Nicobar Islands,1403.0,12.0
1,24/04/2020,Andaman and Nicobar Islands,2679.0,27.0
2,27/04/2020,Andaman and Nicobar Islands,2848.0,33.0
3,01/05/2020,Andaman and Nicobar Islands,3754.0,33.0
4,16/05/2020,Andaman and Nicobar Islands,6677.0,33.0
...,...,...,...,...
2768,30/06/2020,West Bengal,488038.0,18559.0
2769,01/07/2020,West Bengal,497596.0,19170.0
2770,02/07/2020,West Bengal,508001.0,19819.0
2771,03/07/2020,West Bengal,519054.0,20488.0


In [81]:
t_per = [(tests_state_wise["Positive"][i]/tests_state_wise["Total Tested"][i]) for i in range(len(tests_state_wise))]

t_per_positive = [round((tests_state_wise["Total Tested"][i]/tests_state_wise["Positive"][i]),0) for i in range(len(tests_state_wise))]

for i in range(len(t_per)):
    t_per[i] = "{:.2%}".format(t_per[i])
tests_state_wise["Tests Positivity Rate"] = t_per

tests_state_wise["Tests Per Positive Case"] = t_per_positive

tests_state_wise

,Updated On,State,Total Tested,Positive,Tests Positivity Rate,Tests Per Positive Case
0,17/04/2020,Andaman and Nicobar Islands,1403.0,12.0,0.86%,117.0
1,24/04/2020,Andaman and Nicobar Islands,2679.0,27.0,1.01%,99.0
2,27/04/2020,Andaman and Nicobar Islands,2848.0,33.0,1.16%,86.0
3,01/05/2020,Andaman and Nicobar Islands,3754.0,33.0,0.88%,114.0
4,16/05/2020,Andaman and Nicobar Islands,6677.0,33.0,0.49%,202.0
...,...,...,...,...,...,...
2768,30/06/2020,West Bengal,488038.0,18559.0,3.80%,26.0
2769,01/07/2020,West Bengal,497596.0,19170.0,3.85%,26.0
2770,02/07/2020,West Bengal,508001.0,19819.0,3.90%,26.0
2771,03/07/2020,West Bengal,519054.0,20488.0,3.95%,25.0


In [92]:
grp = tests_state_wise.groupby("State")
for i in tests_state_wise["State"].unique():
    group = grp.get_group(i)
    group.drop(columns="State",axis=1,inplace=True)
    group.to_csv('./test_data/state_test_data/'+i+'.csv',index=False)

In [93]:
df = pd.read_csv('./test_data/state_test_data/Karnataka.csv')

In [94]:
df.tail(10)

,Updated On,Total Tested,Positive,Tests Positivity Rate,Tests Per Positive Case
80,25/06/2020,553325.0,10560.0,1.91%,52.0
81,26/06/2020,568058.0,11005.0,1.94%,52.0
82,27/06/2020,581635.0,11923.0,2.05%,49.0
83,28/06/2020,595470.0,13190.0,2.22%,45.0
84,29/06/2020,605159.0,14295.0,2.36%,42.0
85,30/06/2020,620747.0,15242.0,2.46%,41.0
86,01/07/2020,637417.0,16514.0,2.59%,39.0
87,02/07/2020,653627.0,18016.0,2.76%,36.0
88,03/07/2020,671934.0,19710.0,2.93%,34.0
89,04/07/2020,689526.0,21549.0,3.13%,32.0


In [85]:
# for i in tests_state_wise["State"].unique():
#     csvfile = open("./test_data/state_test_data/"+i+".csv", 'r')
#     df = pd.read_csv(csvfile)
#     reader = csv.DictReader( csvfile )
#     mongo_client=MongoClient() 
# #     print(mongo_client[i])
# #     db=mongo_client.Karnataka
# #     db.segment.drop()
#     header = df.columns 

#     for each in reader:
#         row={}
#         for field in header:
#             row[field]=each[field]

#         db.segment.insert(row)

In [98]:
i

'West Bengal'

In [97]:
csvfile = open('./test_data/state_test_data/Karnataka.csv', 'r')
reader = csv.DictReader( csvfile )
mongo_client=MongoClient() 
db=mongo_client.ka
db.segment.drop()
header = df.columns 

for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    db.segment.insert(row)